In [1]:
import numpy as np
import pandas as pd
import math

features = pd.read_csv('./features.csv', index_col='match_id')

# Градиентный бустинг

### 1. Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?

In [2]:
counts = features.count()
max_count = max(counts)
print("Названия признаков, имеющих пропуски: ")
for i, count in enumerate(counts):
    if count < max_count:
        print(features.columns[i])

Названия признаков, имеющих пропуски: 
first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time


Признаки first_blood_time, first_blood_team, first_blood_player1, first_blood_player2 могут быть пропущенные, потому что не в каждой игре происходит первое убийство до 5 минуты. Тоже самое для остальных признаков: покупка курьера, ботла или вардов, также может не произойти за это время.

In [3]:
features = features.fillna(0)

### 2. Как называется столбец, содержащий целевую переменную?

radiant_win

In [4]:
X = features.values[:,:-6]
y = features.values[:, -5]

In [6]:
import time
import datetime
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

selector = KFold(n_splits=5, shuffle=True, random_state=241)
clf = GradientBoostingClassifier(n_estimators=250, random_state=241)

scores = []
times = [] 

for i in range(10,51,5):
    clf = GradientBoostingClassifier(n_estimators=i, random_state=241)
    
    sum_score = 0
    start_time = datetime.datetime.now()
    for train_index, test_index in selector.split(X):
        clf.fit(X[train_index],y[train_index])               
        pred = clf.predict_proba(X[test_index])[:, 1]        
        score = roc_auc_score(y[test_index], pred)
        sum_score += score
        
    time = datetime.datetime.now() - start_time
    times.append(time)
    
    scores.append(sum_score / 5)

for i, s in enumerate(scores):
    print(f"Количество деревьев: {(i + 2) * 5}; качество: {s}, время: {times[i].seconds}")



Количество деревьев: 10; качество: 0.6643877206345742, время: 34
Количество деревьев: 15; качество: 0.6757420237293614, время: 52
Количество деревьев: 20; качество: 0.6828535735340822, время: 64
Количество деревьев: 25; качество: 0.6868481585275605, время: 87
Количество деревьев: 30; качество: 0.6894962060591201, время: 96
Количество деревьев: 35; качество: 0.6919738231779972, время: 112
Количество деревьев: 40; качество: 0.6941311214730337, время: 132
Количество деревьев: 45; качество: 0.6958955138922052, время: 139
Количество деревьев: 50; качество: 0.6974548316948366, время: 153


### 3. Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Какое качество при этом получилось?

In [7]:
print(f"Кросс-валидация для 30 деревьев проводилась - {times[4].seconds} секунд. При этом получилось качество {scores[4]:.3}.")

Кросс-валидация для 30 деревьев проводилась - 96 секунд. При этом получилось качество 0.689.


### 4.Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?

Использование более 30 деревьев в градиентом бустинге не имеет особого смысла, так как возможно переобучение. Чтобы ускорить обучение, можно уменьшит максмимальную глубину деревьев.

# Логистическая регрессия

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)

In [9]:
from sklearn.linear_model import LogisticRegression

scores = []
times = []
coefs = np.power(10.0, np.arange(-5, 4))

for i in coefs:
    clf = LogisticRegression(C=i, random_state=241)
    
    sum_score = 0
    start_time = datetime.datetime.now()
    
    for train_index, test_index in selector.split(X):
        clf.fit(scaled_X[train_index],y[train_index])               
        pred = clf.predict_proba(scaled_X[test_index])[:, 1]        
        score = roc_auc_score(y[test_index], pred)
        sum_score += score
        
    time = datetime.datetime.now() - start_time
    times.append(time)
    
    scores.append(sum_score / 5)
    
for i, s in enumerate(scores):
    print(f"С: {coefs[i]}; качество: {s}, время: {times[i].seconds}")

С: 1e-05; качество: 0.695120379847076, время: 3
С: 0.0001; качество: 0.7112501143920594, время: 5
С: 0.001; качество: 0.7161802463683578, время: 10
С: 0.01; качество: 0.716341462186996, время: 13
С: 0.1; качество: 0.7163100836533356, время: 14
С: 1.0; качество: 0.716306583645544, время: 14
С: 10.0; качество: 0.7163063399602339, время: 14
С: 100.0; качество: 0.7163062657792337, время: 14
С: 1000.0; качество: 0.7163062636530346, время: 14


### 1. Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

Наилучшее качество 0.716. Немного лучше чем у градиентного бустинга, потому что линейные методы работают лучше с большим числом признаков. Работает гораздо быстрее.

### 2.Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?

In [10]:
scaled_X_nocat = scaled_X
for i in range(9,-1,-1):
    scaled_X_nocat = np.delete(scaled_X_nocat, 2 + i * 8, 1)
scaled_X_nocat = np.delete(scaled_X_nocat, 1, 1)

In [11]:
scores = []
times = []

for i in coefs:
    clf = LogisticRegression(C=i, random_state=241)
    
    sum_score = 0
    start_time = datetime.datetime.now()
    
    for train_index, test_index in selector.split(X):
        clf.fit(scaled_X_nocat[train_index],y[train_index])               
        pred = clf.predict_proba(scaled_X_nocat[test_index])[:, 1]        
        score = roc_auc_score(y[test_index], pred)
        sum_score += score
        
    time = datetime.datetime.now() - start_time
    times.append(time)
    
    scores.append(sum_score / 5)
    
for i, s in enumerate(scores):
    print(f"С: {coefs[i]}; качество: {s}, время: {times[i].seconds}")

С: 1e-05; качество: 0.6950569329910983, время: 2
С: 0.0001; качество: 0.7112483906159717, время: 4
С: 0.001; качество: 0.7162355910206267, время: 9
С: 0.01; качество: 0.7164009506527343, время: 12
С: 0.1; качество: 0.7163737844721112, время: 13
С: 1.0; качество: 0.7163707526581122, время: 13
С: 10.0; качество: 0.7163704793048005, время: 13
С: 100.0; качество: 0.7163704962706654, время: 13
С: 1000.0; качество: 0.7163705301659756, время: 13


Качество практически не изменилось (0.716). Связано с маленькими весами у категориальных признаков.

### 3. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [12]:
heroes = []
heroes = np.union1d(features["r1_hero"].values, heroes)
heroes = np.union1d(features["r2_hero"].values, heroes)
heroes = np.union1d(features["r3_hero"].values, heroes)
heroes = np.union1d(features["r4_hero"].values, heroes)
heroes = np.union1d(features["r5_hero"].values, heroes)
heroes = np.union1d(features["d1_hero"].values, heroes)
heroes = np.union1d(features["d2_hero"].values, heroes)
heroes = np.union1d(features["d3_hero"].values, heroes)
heroes = np.union1d(features["d4_hero"].values, heroes)
heroes = np.union1d(features["d5_hero"].values, heroes)

In [13]:
print(f"Различных идентификаторов героев: {heroes.size}")

Различных идентификаторов героев: 108


In [14]:
N = np.max(features.r2_hero.unique())
X_pick = np.zeros((features.shape[0], N))

for i, match_id in enumerate(features.index):
    for p in range(5):
        X_pick[i, features.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
        

C:\Users\Dmitriy\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [15]:
X_pick_all = np.concatenate((scaled_X_nocat,X_pick),axis=1)

### 4. Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?

In [16]:
scores = []
times = []

for i in coefs:
    clf = LogisticRegression(C=i, random_state=241)
    
    sum_score = 0
    start_time = datetime.datetime.now()
    
    for train_index, test_index in selector.split(X):
        clf.fit(X_pick_all[train_index],y[train_index])               
        pred = clf.predict_proba(X_pick_all[test_index])[:, 1]        
        score = roc_auc_score(y[test_index], pred)
        sum_score += score
        
    time = datetime.datetime.now() - start_time
    times.append(time)
    
    scores.append(sum_score / 5)
    
for i, s in enumerate(scores):
    print(f"С: {coefs[i]}; качество: {s}, время: {times[i].seconds}")

    

С: 1e-05; качество: 0.6991713216855027, время: 3
С: 0.0001; качество: 0.725022149276278, время: 5
С: 0.001; качество: 0.7462962333706958, время: 11
С: 0.01; качество: 0.7517359843797606, время: 19
С: 0.1; качество: 0.751937474938148, время: 26
С: 1.0; качество: 0.7519195723934251, время: 27
С: 10.0; качество: 0.7519170090246463, время: 27
С: 100.0; качество: 0.7519172230074844, время: 27
С: 1000.0; качество: 0.7519170387747672, время: 27


 При добавлении "мешка слов" качество улучшилось с 0.716 до 0.752. Это можно обЪянить тем, что при использовании категориальных признаков напрямую, они не несут никакой полезной информации для классификатора. 

### 5. Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

In [17]:
features_test = pd.read_csv('./features_test.csv', index_col='match_id')
features_test = features_test.fillna(0)
X_test = features_test.values

In [18]:
X_test_nocat = X_test
for i in range(9,-1,-1):
    X_test_nocat = np.delete(X_test_nocat, 2 + i * 8, 1)
X_test_nocat =  np.delete(X_test_nocat, 1, 1)

N = np.max(features_test.r2_hero.unique())
X_test_pick = np.zeros((features_test.shape[0], N))

for i, match_id in enumerate(features_test.index):
    for p in range(5):
        X_test_pick[i, features_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_test_pick[i, features_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
X_test_pick_all = np.concatenate((X_test_nocat,X_test_pick),axis=1)
X_test_pick_all_scaled = scaler.fit_transform(X_test_pick_all)

C:\Users\Dmitriy\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()


In [19]:
clf = LogisticRegression(C=1, random_state=241)
clf.fit(X_pick_all,y)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=241, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [20]:
result = clf.predict_proba(X_test_pick_all_scaled)

In [22]:
print(f"Максимальное значение: {max(result[:,1]):.3}, минимальное значение: {min(result[:,1]):.3}")

Максимальное значение: 1.0, минимальное значение: 1.3e-05
